<a href="https://colab.research.google.com/github/KBVKarthik/Tensorflow-Practice/blob/main/Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
import tensorflow as tf
from tensorflow import keras

In [130]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 96 kB 2.7 MB/s 


In [131]:
import keras_tuner as kt

In [132]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [133]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [134]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))


  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [135]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [136]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [137]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])


best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 53s]
val_accuracy: 0.8769999742507935

Best val_accuracy So Far: 0.8893333077430725
Total elapsed time: 00h 16m 07s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit



The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 512 and the optimal learning rate for the optimizer
is 0.001.



In [138]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.4922 - accuracy: 0.8233 - val_loss: 0.3886 - val_accuracy: 0.8609
Epoch 2/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3687 - accuracy: 0.8658 - val_loss: 0.3586 - val_accuracy: 0.8681
Epoch 3/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3319 - accuracy: 0.8782 - val_loss: 0.3424 - val_accuracy: 0.8789
Epoch 4/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3055 - accuracy: 0.8878 - val_loss: 0.3402 - val_accuracy: 0.8790
Epoch 5/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2843 - accuracy: 0.8940 - val_loss: 0.3341 - val_accuracy: 0.8821
Epoch 6/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2686 - accuracy: 0.9004 - val_loss: 0.3153 - val_accuracy: 0.8877
Epoch 7/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2549 - accuracy: 0.9053 - val_loss: 0.3268 - val_accuracy:

In [139]:
hypermodel = tuner.hypermodel.build(best_hps)


hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/47
1500/1500 [==============================] - 10s 6ms/step - loss: 0.4919 - accuracy: 0.8260 - val_loss: 0.3893 - val_accuracy: 0.8583
Epoch 2/47
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3707 - accuracy: 0.8647 - val_loss: 0.3498 - val_accuracy: 0.8711
Epoch 3/47
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3324 - accuracy: 0.8788 - val_loss: 0.3613 - val_accuracy: 0.8721
Epoch 4/47
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3060 - accuracy: 0.8876 - val_loss: 0.3424 - val_accuracy: 0.8753
Epoch 5/47
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2849 - accuracy: 0.8952 - val_loss: 0.3282 - val_accuracy: 0.8819
Epoch 6/47
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2703 - accuracy: 0.8997 - val_loss: 0.3465 - val_accuracy: 0.8768
Epoch 7/47
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2576 - accuracy: 0.9037 - val_loss: 0.3270 - val_accuracy

In [140]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 4ms/step - loss: 0.5653 - accuracy: 0.8906
[test loss, test accuracy]: [0.5653303861618042, 0.8906000256538391]
